# Proportional mapping of S and partial synchronization of G1 model

In [1]:
# required libraries
import numpy as np
import os,sys
import pandas as pd
import time
import random

In [2]:
long_cell_cycle_length_mean = 340
long_cell_cycle_length_std = 32
short_cell_cycle_length_mean = 119
short_cell_cycle_length_std = 10
cell_diameter = 13.2

In [3]:
def cellDivisions(tm,td,ts,stem,l,positions,tau):
    
    # A cell divides when it completes its cell cycle,
    # meaning that the time remaining to cell division ('tr') reachs zero.
    
    global n # current number of cell in the tissue
    
    for dividingCell in range(0,n):
        tc = ts - tm[dividingCell] # 'tc' is elapsed time since last division ('tm'), while 'ts' is the simulation time
        tr = td[dividingCell] - tc # 'tr' is the remaining time for cell division, while 'td' is the cell cycle length
        if (dividingCell-n0)*cell_diameter > l and ts == tau:    # Pulse
            M_G2 = np.random.normal(9,6)
            S_noSkip = np.random.normal(88,9)
            S_skip = np.random.normal(91,12)#S_skip = np.random.normal(91,30)
            G1_noSkip = np.random.normal(22,19)
            G1_skip = np.random.normal(130,35)#G1_skip = np.random.normal(130,73)
            if tr > M_G2:    
            # if in G2/M keeps going
                if M_G2 <= tr < M_G2+S_noSkip+S_skip:
                # if in S
                    td[dividingCell] = M_G2+((tr-M_G2)*S_noSkip)/(S_noSkip+S_skip) # mapping
                    tm[dividingCell] = ts
                elif M_G2+S_noSkip+S_skip <= tr < M_G2+S_noSkip+S_skip+G1_noSkip:      
                # if in G1
                    td[dividingCell] = td[dividingCell]-S_skip # shortens cell cycle subtracting S phase skip
                else:     
                # if in G1 skip
                    td[dividingCell] = M_G2+S_noSkip+G1_noSkip # avoids G1 skip and subtracts S phase skip
                    tm[dividingCell] = ts
            stem[dividingCell] = True   # if the cell is recruited cell is marked as "stem"
        if tr <= 0:
        # if the cell completes its cell cycle, duplicates all cell properties array
            tm[dividingCell] = ts
            tm = np.insert(tm,dividingCell,ts)
            tm = tm[:-1]            
            n = n+1 # increase the current number of cells in the tissue by 1
            positions = np.insert(positions,dividingCell,-1)
            if stem[dividingCell]:
            # daughters of recruited cells are also recruited cells                
                td[dividingCell] = np.random.normal(short_cell_cycle_length_mean, short_cell_cycle_length_std)
                td = np.insert(td,dividingCell,np.random.normal(short_cell_cycle_length_mean, short_cell_cycle_length_std))
                td = td[:-1]
                stem = np.insert(stem,dividingCell,True)
                stem = stem[:-1]
            else:
            # daughters of non-recruited cells are also non-recruited cells                
                td[dividingCell] = np.random.normal(long_cell_cycle_length_mean, long_cell_cycle_length_std)
                td = np.insert(td,dividingCell,np.random.normal(long_cell_cycle_length_mean, long_cell_cycle_length_std))
                td = td[:-1]
                stem = np.insert(stem,dividingCell,False)
                stem = stem[:-1]                
    return tm,td,stem,positions

In [4]:
def run(steps,tm,td,stem,l,positions,tau):
    
    # calls 'cellDivisions' each step and makes time goes by

    global n
    outgrowth = []
    tracking = []
    tracking.append(positions)
    for ts in range(0,steps+1,1):
        tm,td,stem,positions = cellDivisions(tm,td,ts,stem,l,positions,tau)
        outgrowth.append(n-n0)    # Appends the outgrowth for each time
        tracking.append(positions)
    return outgrowth,tracking

In [5]:
n0 = 200  # Number of cells after amputation
l = -1050  # Recruitment limit
tau = 24*8 # Delay

root = 'simulations/'
model = 'mixed/'    
folder = 'n0='+str(n0)+'_l='+str(l)+'_tau='+str(tau)+'/'
if not os.path.isdir(root+model+folder):
    os.makedirs(root+model+folder)
    os.makedirs(root+model+folder+'outgrowth/')
    os.makedirs(root+model+folder+'trajectories/')
    
np.random.seed(1)

emptySpace = 250
lattice = n0+emptySpace
steps = 24*8  # Number of steaps (in hours)

print('n0:',n0)
print('l:',l)
print('tau:',tau)
print('-------')
for seed in range(0,1000):
# for each seed reset the number of cells, initialize cell properties, calls run and saves output            
    n = n0
    positions = np.arange(0,n0) 
    tm = np.random.uniform(-(long_cell_cycle_length_mean-2*long_cell_cycle_length_std),0,lattice)
    td = np.random.normal(long_cell_cycle_length_mean,long_cell_cycle_length_std,lattice)
    stem = np.zeros(lattice)
    solution,track = run(steps,tm,td,stem,l,positions,tau)

    # saves outgrowth
    d1 = {"time":np.arange(0,steps+1),"outgrowth":solution}
    df1 = pd.DataFrame(d1)
    outfile1 = open(root+model+folder+'outgrowth'+'/'+'sim_seed='+str(seed)+'.csv', 'a')
    df1.to_csv(outfile1, sep=',')
    outfile1.close()

    # saves trajectories
    d2 = {"positions":track}
    df2 = pd.DataFrame(d2)
    outfile2 = open(root+model+folder+'trajectories'+'/'+'trajectories_sim_seed='+str(seed)+'.csv', 'a')
    df2.to_csv(outfile2, sep=',')
    outfile2.close() 

n0: 200
l: -1050
tau: 192
-------
